In [6]:
import os
import sys
sys.path.append(os.path.abspath(".."))
import pandas as pd
import numpy as np
import time
from preprocessing import *
from HDEM import *

# set_seed(42)

In [7]:
df = pd.read_csv(r'..\output_csv\HCMUT-SuperNodeXP-2017-1.0.swf.csv')
# df = df.head(100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   job_id                    15886 non-null  float64
 1   submit_time               15886 non-null  float64
 2   wait_time                 15886 non-null  float64
 3   run_time                  15886 non-null  float64
 4   num_allocated_processors  15886 non-null  float64
 5   avg_cpu_time_used         15886 non-null  float64
 6   used_memory               15886 non-null  float64
 7   requested_processors      15886 non-null  float64
 8   requested_time            15886 non-null  float64
 9   requested_memory          15886 non-null  float64
 10  status                    15886 non-null  float64
 11  user_id                   15886 non-null  float64
 12  group_id                  15886 non-null  float64
 13  executable_id             15886 non-null  float64
 14  queue_

In [8]:
df.head()

,job_id,submit_time,wait_time,run_time,num_allocated_processors,avg_cpu_time_used,used_memory,requested_processors,requested_time,requested_memory,status,user_id,group_id,executable_id,queue_id,partition_id,preceding_job_id,think_time
0,4.0,0.0,0.0,13.0,24.0,7.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
1,8.0,40819.0,0.0,61.0,24.0,45.0,14761.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
2,10.0,45255.0,0.0,60.0,24.0,45.0,14760.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
3,11.0,45782.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0
4,12.0,56510.0,0.0,1.0,24.0,0.0,0.0,24.0,21600.0,218453.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0


In [9]:
feature_columns = ['requested_processors', 'requested_time', 'avg_cpu_time_used', 'used_memory', 'submit_time', 'wait_time', 'user_id', 'group_id', 'executable_id', 'queue_id']
target_column = 'run_time'

X_train, X_val, X_test, Y_train, Y_val, Y_test, scaler = prepare_data_DL(df, feature_columns, target_column)

In [10]:
rmse_lst, mae_lst, r2_lst, infer_time_lst = [], [], [], []
for iter in range(100):
    print(f"Iteration {iter + 1} / 100")
    HDEM = Dynamic_Weighted_Ensemble(X_train, X_val, X_test, Y_train, Y_val, Y_test, scaler)

    HDEM.meta_model_name = 'gradientboosting'
    # HDEM.meta_model_name = 'randomforest'
    model_combinations = [['extratrees', 'randomforest', 'xgboost'], ['randomforest', 'mlp', 'gradientboosting'], ['lasso', 'xgboost', 'extratrees']]
    HDEM.num_sub = len (model_combinations)
    HDEM.init_base_sub(model_combinations)
    sub_ensemble_result = HDEM.run_model()
    rmse, mae, r2, infer_time = sub_ensemble_result['RMSE'], sub_ensemble_result['MAE'], sub_ensemble_result['R² Score'], sub_ensemble_result['Inference Time']
    rmse_lst.append(rmse)
    mae_lst.append(mae)
    r2_lst.append(r2)
    infer_time_lst.append(infer_time)

Iteration 1 / 100
Key on Test Set
Predict on Test Set
MAE:  11204.946283888186
RMSE:  28505.711078155797
R2:  0.9536950563659622
Inference Time:  0.00093815007875132
Iteration 2 / 100
Key on Test Set
Predict on Test Set
MAE:  10753.35567822165
RMSE:  29752.25170719981
R2:  0.9495567243218387
Inference Time:  0.0008214434912038403
Iteration 3 / 100
Key on Test Set
Predict on Test Set
MAE:  10439.526830730698
RMSE:  29227.62455337413
R2:  0.9513199919084798
Inference Time:  0.0008460047633148903
Iteration 4 / 100
Key on Test Set
Predict on Test Set
MAE:  11321.053394206505
RMSE:  31301.021397482837
R2:  0.9441683296948277
Inference Time:  0.0008257322533186092
Iteration 5 / 100
Key on Test Set
Predict on Test Set
MAE:  10615.366924151316
RMSE:  29983.858320261676
R2:  0.9487683154625541
Inference Time:  0.0008295573467432066
Iteration 6 / 100
Key on Test Set
Predict on Test Set
MAE:  11058.913745127764
RMSE:  28984.44954788549
R2:  0.9521266613584587
Inference Time:  0.000837540903756784

In [12]:
# Save results to CSV
results_df = pd.DataFrame({
    'RMSE': rmse_lst,
    'MAE': mae_lst,
    'R2': r2_lst,
    'Inference Time': infer_time_lst
})
results_df.to_csv('../output_HCMUT/HDEM_results.csv', index=False)